In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype

In [2]:
df = pd.read_csv("data/yellow_tripdata_combined_cleaned_focus.csv")

In [3]:
df.head()

,Unnamed: 0,trip_distance,PULocationID,DOLocationID,fare_amount,tip_amount,total_amount,pickup_at,dropoff_at,duration
0,0,3.80,142,236,14.5,3.65,21.95,2022-01-01 00:35:40,2022-01-01 00:53:29,1069.0
1,1,2.10,236,42,8.0,4.00,13.30,2022-01-01 00:33:43,2022-01-01 00:42:07,504.0
2,2,0.97,166,166,7.5,1.76,10.56,2022-01-01 00:53:21,2022-01-01 01:02:19,538.0
3,3,1.09,114,68,8.0,0.00,11.80,2022-01-01 00:25:21,2022-01-01 00:35:23,602.0
4,4,4.30,68,163,23.5,3.00,30.30,2022-01-01 00:36:48,2022-01-01 01:14:20,2252.0


In [4]:
df["pickup_at"] = pd.to_datetime(df["pickup_at"])
df["dropoff_at"] = pd.to_datetime(df["dropoff_at"])
df["pickup_at_hour"] = df["pickup_at"].dt.floor("H")
df["duration"] = df["duration"]/60

In [5]:
df_weather = pd.read_csv("data/weather_hourly.csv", index_col=False)
df_weather["time"] = pd.to_datetime(df_weather["time"])

In [6]:
df_weather.head()

,time,precipitation (mm)
0,2022-01-01 00:00:00,0.0
1,2022-01-01 01:00:00,0.0
2,2022-01-01 02:00:00,0.0
3,2022-01-01 03:00:00,0.7
4,2022-01-01 04:00:00,0.0


In [7]:
# Merge trips data (df) with weather data for that particular hour
# (df_weather) on the "time" column.
#
#small["pickup_at"].dt.round("H")
merged = pd.merge(df, df_weather, left_on="pickup_at_hour", right_on="time")

In [8]:
# Make new column "weather" that is "rainy" if precipitation > 0, else "sunny".

merged["weather"] = "sunny"
merged["weather"][merged["precipitation (mm)"] > 0] = "rainy"

C:\Users\anned\AppData\Local\Temp\ipykernel_8620\4246521071.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged["weather"][merged["precipitation (mm)"] > 0] = "rainy"


In [9]:
# day/night, sunny/rainy, spring/summer/fall/winter, area (manhattan/all)
merged["time"] = "day"
merged["time"][20 < merged["pickup_at"].dt.hour] = "night"
merged["time"][merged["pickup_at"].dt.hour < 6] = "night"

C:\Users\anned\AppData\Local\Temp\ipykernel_8620\2682998250.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged["time"][20 < merged["pickup_at"].dt.hour] = "night"
C:\Users\anned\AppData\Local\Temp\ipykernel_8620\2682998250.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged["time"][merged["pickup_at"].dt.hour < 6] = "night"


In [10]:
merged["pickup_area"] = "not manhattan"
merged["dropoff_area"] = "not manhattan"
manhattan_location_ids = [103, 104, 105, 12, 88, 87, 209, 261, 13, 45, 231, 232, 148, 144, 211, 125, 4, 79, 114, 113, 249, 158, 224, 107, 234, 90, 68, 246, 186, 164, 137, 170, 233, 100, 229, 162, 161, 230, 48, 50, 202, 140, 141, 237, 142, 143, 43, 262, 263, 236, 239, 238, 75, 151, 24, 74, 41, 166, 194, 42, 152, 116, 244, 120, 243, 127, 128, 153]
merged["pickup_area"][merged["PULocationID"].isin(manhattan_location_ids)] = "manhattan"
merged["dropoff_area"][merged["DOLocationID"].isin(manhattan_location_ids)] = "manhattan"

C:\Users\anned\AppData\Local\Temp\ipykernel_8620\3562136131.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged["pickup_area"][merged["PULocationID"].isin(manhattan_location_ids)] = "manhattan"
C:\Users\anned\AppData\Local\Temp\ipykernel_8620\3562136131.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged["dropoff_area"][merged["DOLocationID"].isin(manhattan_location_ids)] = "manhattan"


In [11]:
merged["season"] = "winter"
merged["season"][(merged["pickup_at"].dt.month > 2) & (merged["pickup_at"].dt.month < 6)] = "spring"
merged["season"][(merged["pickup_at"].dt.month > 5) & (merged["pickup_at"].dt.month < 9)] = "summer"
merged["season"][(merged["pickup_at"].dt.month > 8) & (merged["pickup_at"].dt.month < 12)] = "fall"

C:\Users\anned\AppData\Local\Temp\ipykernel_8620\3454911036.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged["season"][(merged["pickup_at"].dt.month > 2) & (merged["pickup_at"].dt.month < 6)] = "spring"
C:\Users\anned\AppData\Local\Temp\ipykernel_8620\3454911036.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged["season"][(merged["pickup_at"].dt.month > 5) & (merged["pickup_at"].dt.month < 9)] = "summer"
C:\Users\anned\AppData\Local\Temp\ipykernel_8620\3454911036.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http

In [12]:
focus = ['trip_distance','fare_amount','tip_amount','total_amount','duration']

In [13]:
dayname_type = CategoricalDtype(categories=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"], ordered=True)
by_dayname = merged.groupby([
    merged["pickup_at"].dt.day_name().astype(dayname_type),
    merged["weather"], 
    merged["season"], 
    merged["time"], 
    merged["pickup_area"]
])

In [14]:
day_out = pd.concat((
    by_dayname[focus].sum(), 
    by_dayname["duration"].count() # Don't need to choose an attribute here, since we're just counting the number of rows.
), axis=1)
day_out.columns = focus + [ "number_of_trips"]
day_out = day_out.loc[day_out['number_of_trips']!=0]
day_out.to_csv("output/day_out.csv")

In [15]:
monthname_type = CategoricalDtype(categories=["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"], ordered=True)
by_monthname = merged.groupby([
    merged["pickup_at"].dt.month_name().astype(monthname_type),
    merged["weather"], 
    merged["season"], 
    merged["time"], 
    merged["pickup_area"]
])

In [16]:
month_out = pd.concat((
    by_monthname[focus].sum(), 
    by_monthname["duration"].count() # Don't need to choose an attribute here, since we're just counting the number of rows.
), axis=1)
month_out.columns = focus + [ "number_of_trips"]
month_out = month_out.loc[month_out['number_of_trips']!=0]
month_out.to_csv("output/month_out.csv")

In [17]:
by_zones = merged.groupby([
    merged["PULocationID"],
    merged["DOLocationID"],
    merged["weather"], 
    merged["season"], 
    merged["time"], 
    merged["pickup_area"]
])

In [18]:
zones_out = pd.concat((
    by_zones[focus].sum(), 
    by_zones["duration"].count() # Don't need to choose an attribute here, since we're just counting the number of rows.
), axis=1)
zones_out.columns = focus + [ "number_of_trips"]
zones_out = zones_out.loc[zones_out['number_of_trips']!=0]
zones_out.to_csv("output/zones_out.csv")

In [19]:
by_PUzone = merged.groupby([
    merged["PULocationID"],
    merged["weather"], 
    merged["season"], 
    merged["time"], 
    merged["pickup_area"]
])

In [20]:
PUzone_out = pd.concat((
    by_PUzone[focus].sum(), 
    by_PUzone["duration"].count() # Don't need to choose an attribute here, since we're just counting the number of rows.
), axis=1)
PUzone_out.columns = focus + [ "number_of_trips"]
PUzone_out = PUzone_out.loc[PUzone_out['number_of_trips']!=0]
PUzone_out.to_csv("output/PUzones_out.csv")

In [21]:
by_DOzone = merged.groupby([
    merged["DOLocationID"],
    merged["weather"], 
    merged["season"], 
    merged["time"], 
    merged["pickup_area"]
])

In [22]:
DOzone_out = pd.concat((
    by_DOzone[focus].sum(), 
    by_DOzone["duration"].count() # Don't need to choose an attribute here, since we're just counting the number of rows.
), axis=1)
DOzone_out.columns = focus + [ "number_of_trips"]
DOzone_out = DOzone_out.loc[DOzone_out['number_of_trips']!=0]
DOzone_out.to_csv("output/DOzones_out.csv")